# Modelling

In [1]:
import tensorflow as tf
import pandas as pd
from keras import models, layers, losses, optimizers, metrics, callbacks

from keras_preprocessing.image import ImageDataGenerator

%run ../scripts/save_utils.py

## Baseline model

In [2]:
x_train, y_train, x_val, y_val, x_test, y_test = load_data('../save_files/processed_data.pkl')

In [3]:
df_train = pd.DataFrame({'image_path':x_train, 'label':y_train})
df_val = pd.DataFrame({'image_path':x_val, 'label':y_val})
df_test = pd.DataFrame({'image_path':x_test, 'label':y_test})

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_dataframe(df_train, '..\\data\\raw\\merged_data\\',
                                                    x_col='image_path', y_col='label',
                                                    target_size=(512, 512), batch_size=10,
                                                    class_mode='categorical', validate_filenames=False)

validation_generator = val_datagen.flow_from_dataframe(df_val, '..\\data\\raw\\merged_data\\',
                                                       x_col='image_path', y_col='label',
                                                       target_size=(512, 512), batch_size=10,
                                                       class_mode='categorical', validate_filenames=False)

test_generator = test_datagen.flow_from_dataframe(df_test, '..\\data\\raw\\merged_data\\',
                                                  x_col='image_path', y_col='label',
                                                  target_size=(512, 512), batch_size=10,
                                                  class_mode='categorical', validate_filenames=False)

Found 4213 non-validated image filenames belonging to 4 classes.
Found 1405 non-validated image filenames belonging to 4 classes.
Found 1405 non-validated image filenames belonging to 4 classes.


In [6]:
tf.keras.backend.clear_session()

baseline_model = models.Sequential()
baseline_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(512, 512, 3)))
baseline_model.add(layers.MaxPooling2D(2, 2))
baseline_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
baseline_model.add(layers.MaxPooling2D(2, 2))
baseline_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
baseline_model.add(layers.MaxPooling2D(2, 2))
baseline_model.add(layers.Flatten())
baseline_model.add(layers.Dense(256, activation='relu'))
baseline_model.add(layers.Dense(4, activation='softmax'))

In [7]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 126, 128)    0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 128)     147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 62, 62, 128)      0

In [8]:
baseline_model.compile(loss=losses.CategoricalCrossentropy(), optimizer=optimizers.Adam(learning_rate=0.001), metrics=[metrics.Recall()])

In [9]:
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
baseline_model.fit(train_generator, steps_per_epoch=100, epochs=30, validation_data=validation_generator, validation_steps=50, callbacks=[stop_early], verbose=1)

Epoch 1/30
100/100 [==============================] - 28s 183ms/step - loss: 1.3590 - recall: 0.4079 - val_loss: 0.5463 - val_recall: 0.7500
Epoch 2/30
100/100 [==============================] - 17s 165ms/step - loss: 0.6324 - recall: 0.6908 - val_loss: 0.5182 - val_recall: 0.7740
Epoch 3/30
100/100 [==============================] - 17s 165ms/step - loss: 0.4781 - recall: 0.8000 - val_loss: 0.5612 - val_recall: 0.7560
Epoch 4/30
100/100 [==============================] - 16s 162ms/step - loss: 0.3725 - recall: 0.8460 - val_loss: 0.3573 - val_recall: 0.8720
Epoch 5/30
100/100 [==============================] - 15s 148ms/step - loss: 0.2906 - recall: 0.9040 - val_loss: 0.3605 - val_recall: 0.8660
Epoch 6/30
100/100 [==============================] - 15s 149ms/step - loss: 0.2570 - recall: 0.8970 - val_loss: 0.2721 - val_recall: 0.9120
Epoch 7/30
100/100 [==============================] - 15s 148ms/step - loss: 0.2264 - recall: 0.9134 - val_loss: 0.2759 - val_recall: 0.9200
Epoch 8/30
10

In [11]:
baseline_model.save('..\\save_files\\baseline.h5') #weighs too much for upload haha

In [12]:
history = baseline_model.evaluate(test_generator, batch_size=32, return_dict=True)

141/141 [==============================] - 13s 92ms/step - loss: 0.3618 - recall: 0.9317


In [13]:
print('test loss:   ', history['loss'])
print('test recall: ', history['recall'])

test loss:    0.361837774515152
test recall:  0.9316725730895996
